# Stage 3: Achieve higher derivatives

这个阶段的主要目标是扩展 DeZero，使其能够计算高阶导数。

In [8]:
import numpy as np
from dezero import Variable
import os
import subprocess

## Step 25: Visualization of computational graphs Ⅰ

使用Graphviz库，我们可以将计算图可视化为图形。这样可以更直观地理解计算图的结构。

## Step 26: Visualization of computational graphs Ⅱ

In [3]:
# 将Variable实例赋给函数，返回DOT语言编写的表示实例信息的字符串
def _dot_var(v, verbose=False):
    dot_var = '{} [label="{}", color=orange, style=filled]\n'
    name = '' if v.name is None else v.name
    # verbose为True时，添加变量的形状和数据类型
    if verbose and v.data is not None:
        if v.name is not None:
            name += ': '
        name += str(v.shape) + ' ' + str(v.dtype)
    return dot_var.format(id(v), name)  # 使用python内置函数id()获取对象的内存地址

In [5]:
x = Variable(np.random.randn(2, 3))
x.name = 'x'
print(_dot_var(x))
print(_dot_var(x, verbose=True))

1402407802760 [label="x", color=orange, style=filled]

1402407802760 [label="x: (2, 3) float64", color=orange, style=filled]



In [4]:
# 将Function实例赋给函数，返回DOT语言编写的表示实例信息的字符串
def _dot_func(f):
    dot_func = '{} [label="{}", color=lightblue, style=filled, shape=box]\n'
    txt = dot_func.format(id(f), f.__class__.__name__)
    # 用DOT记述函数与输入变量之间的连接，以及函数与输出变量之间的连接
    dot_edge = '{} -> {}\n' 
    for x in f.inputs:
        txt += dot_edge.format(id(x), id(f))
    for y in f.outputs:
        txt += dot_edge.format(id(f), id(y()))  # y是weakref
    return txt

In [6]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1
txt = _dot_func(y.creator)
print(txt)

1404453256520 [label="Add", color=lightblue, style=filled, shape=box]
1402407784328 -> 1404453256520
1402407783624 -> 1404453256520
1404453256520 -> 1404287120264



In [7]:
# 参考backward函数的实现，实现获取计算图的函数
def get_dot_graph(output, verbose=True):
    txt = ''
    funcs = []
    seen_set = set()

    def add_func(f):
        if f not in seen_set:  # 不需要按辈分顺序添加函数
            funcs.append(f)
            seen_set.add(f)

    add_func(output.creator)
    txt += _dot_var(output, verbose)  # 输出变量

    while funcs:
        func = funcs.pop()
        txt += _dot_func(func)  # 函数
        for x in func.inputs:
            txt += _dot_var(x, verbose)  # 输入变量

            if x.creator is not None:
                add_func(x.creator)  # 添加函数

    return 'digraph g {\n' + txt + '}'

In [13]:
def plot_dot_graph(output, verbose=True, to_file='graph.png'):
    dot_graph = get_dot_graph(output, verbose)

    # 保存为dot文件
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
    if not os.path.exists(tmp_dir):  # 创建目录
        os.mkdir(tmp_dir)
    dot_file = os.path.join(tmp_dir, 'tmp.dot')
    with open(dot_file, 'w') as f:
        f.write(dot_graph)

    # 调用Graphviz的dot命令
    ext = os.path.splitext(to_file)[1][1:]  # 获取文件扩展名
    cmd = 'dot {} -T {} -o {}'.format(dot_file, ext, to_file)
    subprocess.run(cmd, shell=True)

    # 在juptyer notebook中显示图像
    try:
        from PIL import Image
        img = Image.open(to_file)
        img.show()
    except ImportError:
        pass

In [11]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1
x0.name = 'x0'
x1.name = 'x1'
y.name = 'y'
plot_dot_graph(y, verbose=False, to_file='add.png')

In [15]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()

x.name = 'x'
y.name = 'y'
z.name = 'z'
plot_dot_graph(z, verbose=False, to_file='goldstein.png')